**import libraries**

In [19]:
import pandas as pd
from operator import itemgetter
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import requests
import plotly.graph_objects as go

**Try out Vader Sentiment Analyzer**

In [2]:
#call analyser object
analyser = SentimentIntensityAnalyzer()

In [3]:
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    return score

In [4]:
sentiment_analyzer_scores("Today is an okay day.")

{'neg': 0.0, 'neu': 0.678, 'pos': 0.322, 'compound': 0.2263}

In [5]:
sentiment_analyzer_scores("Today is an amazing day!")

{'neg': 0.0, 'neu': 0.494, 'pos': 0.506, 'compound': 0.6239}

In [6]:
sentiment_analyzer_scores("Today is an amazing day! Vader is working but it took a while")

{'neg': 0.0, 'neu': 0.803, 'pos': 0.197, 'compound': 0.4003}

In [7]:
sentiment_analyzer_scores("Today kind of sucked.")

{'neg': 0.523, 'neu': 0.477, 'pos': 0.0, 'compound': -0.5095}

**Pull in Live News Data from News API**

In [8]:
url = ('http://newsapi.org/v2/everything?'
       'q=Apple&'
       'from=2020-03-03&'
       'sortBy=popularity&'
       'apiKey=' + input('api key: '))

response = requests.get(url)

api key: 798c7707bbe8404199296c0521a99f61


**Perform Analysis on the News Title**

In [58]:
news = response.json().get('articles')

#add title score:
for item in news:
    item.update({'title_score': sentiment_analyzer_scores(item['title'])})

news_df = pd.DataFrame(news)
news_df.head()

,source,author,title,description,url,urlToImage,publishedAt,content,title_score
0,"{'id': None, 'name': 'Lifehacker.com'}",Brendan Hesse,Buy a Refurbished 2017 or 2018 MacBook Pro for...,There’s an incredible deal on refurbished 2017...,https://lifehacker.com/buy-a-refurbished-2017-...,https://i.kinja-img.com/gawker-media/image/upl...,2020-03-04T18:45:00Z,Theres an incredible deal on refurbished 2017 ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,"{'id': None, 'name': 'Lifehacker.com'}",Micaela Heck,"How to Stop Procrastinating, With Behavior Sci...",Do you put things off ‘til the last minute? Ar...,https://lifehacker.com/how-to-stop-procrastina...,https://i.kinja-img.com/gawker-media/image/upl...,2020-03-03T14:30:00Z,Do you put things off til the last minute? Are...,"{'neg': 0.216, 'neu': 0.784, 'pos': 0.0, 'comp..."
2,"{'id': 'engadget', 'name': 'Engadget'}",Richard Lawler,The Morning After: NASA is taking applications...,"Hey, good morning! You look fabulous. SXSW 202...",https://www.engadget.com/2020/03/03/nasa-is-hi...,https://o.aolcdn.com/images/dims?thumbnail=120...,2020-03-03T12:45:00Z,What will you do with $25?Apple will pay up to...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,"{'id': 'engadget', 'name': 'Engadget'}",Engadget,The Morning After: Amazon's accelerated one-da...,"Hey, good morning! You look fabulous. Amazon i...",https://www.engadget.com/2020/03/04/amazon-one...,https://o.aolcdn.com/images/dims?thumbnail=120...,2020-03-04T12:30:00Z,Goodbye doesn't mean forever.The gadgets that ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
4,"{'id': 'engadget', 'name': 'Engadget'}",Igor Bonifacic,Amazon has a rare sale on official iPhone cases,Apple's first-party iPhone cases tend to cost ...,https://www.engadget.com/2020/03/03/amazon-iph...,https://o.aolcdn.com/images/dims?thumbnail=120...,2020-03-03T18:39:00Z,"iPhone 11\r\nWhen it comes to the iPhone 11, t...","{'neg': 0.0, 'neu': 0.805, 'pos': 0.195, 'comp..."


In [17]:
#other ways to add title score object to news


#add title score way 2 -----------------------:

# def add_title_scoore():
#     for item in news:
#         score = {'title_score' : sentiment_analyzer_scores(item['title'])}
#     return news
# add_title_scoore()


#add title score way 3 -----------------------:

# result = [dict(item, scoreeeeeeeeeeeeeee=sentiment_analyzer_scores(item['title'])) for item in news]
# print(result)


#this works append static object -------------:

# for item in news:
#     item.update({'scooooooooooooooooore': 'baaaaaaaaaaaaaaaaaad'})
# print(news)

**Plot Title Sentiment Score**

In [59]:
x = news_df['title']
y = [d.get('neg') for d in news_df['title_score']]
y1 = [d.get('neu') for d in news_df['title_score']]
y2 = [d.get('pos') for d in news_df['title_score']]
y3 = [d.get('compound') for d in news_df['title_score']]

fig = go.Figure(go.Bar(x=x, y=y, name='Negative'))
fig.add_trace(go.Bar(x=x, y=y1, name='Neutral'))
fig.add_trace(go.Bar(x=x, y=y2, name='Positive'))
fig.add_trace(go.Bar(x=x, y=y3, name='Compound'))

fig.update_layout(barmode='stack')
fig.show()